In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
import time
import warnings
import random, json
import pandas as pd
import datetime
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import ExtraTreesRegressor
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
import xgboost as xgb
from xgboost import plot_importance
from scipy.stats import spearmanr
from pprint import pprint
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from functools import reduce
from scipy import stats

In [ ]:
NAPLAN_category = ['NAPLAN-Numeracy','NAPLAN-Reading', 'NAPLAN-Writting']
def read_data():
    train_data = pd.read_csv("../forML/train_2261.csv")
    test_data = pd.read_csv("../forML/test_2261.csv")
    target_train = train_data.loc[:,NAPLAN_category]
    target_test = test_data.loc[:,NAPLAN_category]
    x_train = train_data.drop(NAPLAN_category,1)
    x_test = test_data.drop(NAPLAN_category,1)
    y_train, y_test = dict(), dict()
    y_train["numeracy"], y_train["reading"], y_train["writing"] = target_train[['NAPLAN-Numeracy']],target_train[['NAPLAN-Reading']],target_train[['NAPLAN-Writting']]
    y_test["numeracy"], y_test["reading"], y_test["writing"] = target_test[['NAPLAN-Numeracy']],target_test[['NAPLAN-Reading']],target_test[['NAPLAN-Writting']]
    return x_train,y_train,x_test,y_test

In [ ]:
x_train,y_train,x_test,y_test = read_data()
numeracy_train = y_train["numeracy"]
numeracy_test = y_test["numeracy"]

In [ ]:
lr = LinearRegression()
maes,rmses,r2s = [],[],[]
preds = []
for _ in range(10):
    lr.fit(x_train, numeracy_train)
    mae,rmse,r2,y_test_pred = evaluate(lr,x_test,numeracy_test)
    evaluate_fairness(y_test_pred,numeracy_test,x_test)

In [ ]:
# Create a based model
rf = RandomForestRegressor()
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1000, num = 3)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 100, num = 5)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4,8]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# Random search of parameters, using 5 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
for category in y_train:
    rf_random.fit(x_train, y_train[category])
    best_random = rf_random.best_estimator_
    print(rf_random.best_params_, rf_random.best_estimator_)

In [ ]:
# Taking numeracy test as an example, after we obtained the initial ranges for hyperparameters, 
# we mannually finr-tuned the paratermeters as follows
param_grid = {
    'bootstrap': [False],
    'max_depth': [25,32,40],
    'max_features': ['sqrt'],
    'min_samples_leaf': [1,2, 3],
    'min_samples_split': [4,5,8],
    'n_estimators': [500, 600, 700]
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)
grid_search.fit(x_train,numeracy_train)
grid_search.best_params_, grid_search.best_score_
#******************************************************Test*************************************************************************
rf = RandomForestRegressor(max_depth=70,max_features='sqrt',min_samples_leaf=1, min_samples_split=3,n_estimators=1000,bootstrap=False,n_jobs=-1)
maes,rmses,r2s = [],[],[]
preds = []
for _ in range(10): 
    rf.fit(x_train,numeracy_train)
    mae,rmse,r2,y_test_pred = evaluate(rf,x_test,numeracy_test,print_info=False)
    evaluate_fairness(y_test_pred,numeracy_test,x_test)

In [ ]:
scaler = MinMaxScaler()
x_train_sc = scaler.fit_transform(x_train)
x_test_sc = scaler.transform(x_test)
param_grid = {
    'hidden_layer_sizes': [(500, 240, 120), (480, 240, 120), (500, 200, 100)],
    'activation': ['identity'],
    'solver': ['adam','lbfgs'],
    'alpha': [0.0001, 0.001, 0.01, 0.1, 1],
    'learning_rate':['constant']
}
mlp = MLPRegressor(random_state=42, max_iter=3000,early_stopping=True,n_iter_no_change=20)
grid = GridSearchCV(mlp, param_grid, n_jobs= -1, cv=5)
grid.fit(x_train_sc,numeracy_train)
print(grid.best_params_) 
#******************************************************Test*************************************************************************
mlp = MLPRegressor(hidden_layer_sizes=(200,100),solver="lbfgs", activation="identity" ,alpha=0.0001,learning_rate='constant',random_state=42, max_iter=3000,early_stopping=True,n_iter_no_change=20)
for _ in range(10): 
    mlp.fit(x_train_sc,numeracy_train.values)
    mae,rmse,r2,y_test_pred = evaluate(mlp,x_test_sc,numeracy_test.values,print_info=False)
    evaluate_fairness(y_test_pred,numeracy_test,x_test)

In [ ]:
def modelfit(alg, X_train, y_train, useTrainCV=True, cv_folds=5, early_stopping_rounds=20):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(X_train, label=y_train)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='mae', early_stopping_rounds=early_stopping_rounds, verbose_eval=False)
#         print(cvresult['test-rmse-mean'].min())
#         print(cvresult['test-rmse-mean'].argmin())
        alg.set_params(n_estimators=cvresult.shape[0])
        #print(cvresult.shape)
    
    #Fit the algorithm on the data
    alg.fit(X_train, y_train,eval_metric='mae')
        
    #Predict training set:
    evaluate(alg,X_train,y_train)

In [ ]:
param_test1 = {
 'max_depth':range(3,15,2),
 'min_child_weight':range(1,14,2)
}
gsearch1 = GridSearchCV(estimator = xgb.XGBRegressor(learning_rate =0.1, n_estimators=1000, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,nthread=4, scale_pos_weight=1, seed=42), 
 param_grid = param_test1,scoring='neg_mean_absolute_error',n_jobs=4,cv=10)
gsearch1.fit(x_train,numeracy_train)
gsearch1.best_params_, gsearch1.best_score_

In [ ]:
param_test2 = {
 'gamma':[i/100.0 for i in range(0,5)]
}
gsearch2 = GridSearchCV(estimator = xgb.XGBRegressor(learning_rate =0.1, n_estimators=1000, max_depth=8,
 min_child_weight=7, gamma=0, subsample=0.8, colsample_bytree=0.8,nthread=4, scale_pos_weight=1, seed=42), 
 param_grid = param_test2,scoring='neg_mean_absolute_error',n_jobs=4,cv=5)
gsearch2.fit(x_train,numeracy_train)
gsearch2.best_params_, gsearch2.best_score_

In [ ]:
param_test3 = {
 'subsample':[i/10.0 for i in range(2,11)],
 'colsample_bytree':[i/10.0 for i in range(2,11)],
  'n_estimators':[200, 400, 600, 800, 1000, 2000] 
}
gsearch3 = GridSearchCV(estimator = xgb.XGBRegressor(learning_rate =0.1, n_estimators=1000, max_depth=8,
 min_child_weight=7, gamma=0.02, subsample=0.8, colsample_bytree=0.8,nthread=4, scale_pos_weight=1, seed=42), 
 param_grid = param_test3,scoring='neg_mean_absolute_error',n_jobs=4,cv=5)
gsearch3.fit(x_train,numeracy_train)
gsearch3.best_params_, gsearch3.best_score_

In [ ]:
param_test4 = {
    'learning_rate': [0.001,0.01,0.1],
 'reg_alpha':[1e-5, 1e-4, 1e-3, 1e-2, 0.1, 1, 100],
 'reg_lambda':[1e-5, 1e-4, 1e-3, 1e-2, 0.1, 1, 100]   
}
gsearch4 = GridSearchCV(estimator = xgb.XGBRegressor(learning_rate =0.1, n_estimators=1000, max_depth=5,
 min_child_weight=4, gamma=0.02, subsample=0.4, colsample_bytree=0.7, nthread=4, scale_pos_weight=1, seed=42), 
 param_grid = param_test4,scoring='neg_mean_absolute_error',n_jobs=4,cv=5)
gsearch4.fit(x_train,numeracy_train)
gsearch4.best_params_, gsearch4.best_score_

In [ ]:
xgb_best = xgb.XGBRegressor(
                 gamma=0.02,                 
                 learning_rate=0.1,
                 max_depth=8,
                 min_child_weight=6,
                 n_estimators=1000,                                                                    
                 colsample_bytree=0.7,
                 subsample=0.4,
                 reg_alpha=0.0001, 
                 reg_lambda=1,
                 seed=42)
for _ in range(10): 
    modelfit(xgb_best, x_train,numeracy_train)
    mae,rmse,r2,y_test_pred = evaluate(xgb_best,x_test,numeracy_test,print_info=True)
    evaluate_fairness(y_test_pred,numeracy_test,x_test)